<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook shows how to log the payload for a model deployed on SageMaker using Watson OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

### 1.1 Installation and authentication

In [51]:
!pip install sagemaker --no-cache | tail -n 1

In [52]:
!pip install ibm-ai-openscale==1.0.429 --no-cache | tail -n 1

### **Action:** Restart the kernel.

Import and initiate.

In [53]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.supporting_classes.enums import InputDataType, ProblemType
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *

#### ACTION: Get Watson OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your Watson OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing Watson OpenScale instance
```
ibmcloud target -g <myResourceGroup>
```
- get details of the instance
```
ibmcloud resource service-instance 'Watson-OpenScale-instance_name'
```

#### Let's define some constants required to set up data mart:

- WATSON_OS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [59]:
WATSON_OS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "*****",
  "apikey": "*****"
}

In [55]:
POSTGRES_CREDENTIALS = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [56]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

Create the schema for data mart.

In [57]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [60]:
client = APIClient(WATSON_OS_CREDENTIALS)

In [61]:
client.version

'1.0.429'

### 1.2 DataMart setup

In [63]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [64]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `SageMaker` machine learning engine

Provide credentials using the following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [65]:
SAGEMAKER_ENGINE_CREDENTIALS = {'access_key': '****',
                   'secret_key': '****',
                   'region_name': '****'} #i.e. us-east-2

In [66]:
binding_uid = client.data_mart.bindings.add('My SageMaker engine', SageMakerMachineLearningInstance(SAGEMAKER_ENGINE_CREDENTIALS))

In [67]:
bindings_details = client.data_mart.bindings.get_details()

In [68]:
client.data_mart.bindings.list()

152fdb6f-4c8c-43cc-b2e4-73cf388844ef,My SageMaker engine,amazon_sagemaker,2019-01-28T16:45:28.500Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

**Note:** Depending on the number of assets, it may take some time.

In [69]:
client.data_mart.bindings.list_assets()

70b2a1a8cb6b8950ffd2b65afb9236de,Breast-cancer-linear-learner-2019-01-28-16-13,2019-01-28T16:19:02.652Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
1e97982799d7bae6a15f5c98f79d601c,GoSales-linear-binary-classifier-2019-01-22-03-04-36,2019-01-22T03:08:37.167Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
b9231df5c2fe47bd5ed5d8a25cd84366,Product-line-linear-learner-2019-01-21-17-56,2019-01-21T18:04:39.969Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
39dbc78bd91473aae1494627e297c9e3,linear-learner-2019-01-16-14-39-18-576,2019-01-16T14:39:18.653Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
86eae234d9e3655e67a1f6c24b5c0aea,xgboost-2019-01-09-09-56-05-050,2019-01-09T09:56:05.180Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
e27ca462d33209d04f326a576bd6c380,xgboost-2019-01-09-03-33-14-252,2019-01-09T03:33:14.391Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
ec8e2b0c24dcc141f0d3e98f7fc3895f,GoSales-linear-binary-classifier-2019-01-07-14-31-14,2019-01-07T14:35:21.457Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
e2bd79e9359e07ce8a679cbebb67ec11,DEMO-linear-DPJC,2018-12-14T12:14:40.485Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
6455e8fd08d6a178695bda476bfd97cb,DEMO-videogames-xgboost-2018-12-13-16-17-39,2018-12-13T16:21:41.249Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
31320550c76c987cb00a3642a83e287e,arsuryan-product-line-mc-linear-2018-12-12-18-23-12,2018-12-12T18:27:58.878Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False
d2c991680ad36cd65886745d656cd440,DEMO-linear-2018-12-06-15-44-46,2018-12-06T16:04:30.073Z,model,,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,False


**Action:** Assign your source_uid to `source_uid` variable below.

In [72]:
source_uid = '70b2a1a8cb6b8950ffd2b65afb9236de'

In [73]:
subscription = client.data_mart.subscriptions.add(
    SageMakerMachineLearningAsset(
                source_uid=source_uid,
                binding_uid=binding_uid,
                input_data_type=InputDataType.STRUCTURED,
                problem_type=ProblemType.MULTICLASS_CLASSIFICATION,
                label_column='diagnosis',
                probability_column='score',
                prediction_column='predicted_label'))

#### Get subscriptions list

In [74]:
subscriptions = client.data_mart.subscriptions.get_details()

In [75]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['70b2a1a8cb6b8950ffd2b65afb9236de']


#### List subscriptions

In [76]:
client.data_mart.subscriptions.list()

70b2a1a8cb6b8950ffd2b65afb9236de,Breast-cancer-linear-learner-2019-01-28-16-13,model,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,2019-01-28T16:46:46.542Z


<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the product line model and measure response time

In [77]:
import requests
import time
import json
import boto3

binding_details = client.data_mart.bindings.get_details(binding_uid=binding_uid)
subscription_details = subscription.get_details()

access_id = binding_details['entity']['credentials']['access_key_id']
access_key = binding_details['entity']['credentials']['secret_access_key']
region = binding_details['entity']['credentials']['region']
endpoint_name = subscription_details['entity']['deployments'][0]['name']
runtime = boto3.client('sagemaker-runtime', region_name=region, aws_access_key_id=access_id, aws_secret_access_key=access_key)

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
        
payload = "17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482\n17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482"

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": 1.0,
      "predicted_label": 1.0
    },
    {
      "score": 1.0,
      "predicted_label": 1.0
    }
  ]
}


### 4.2 Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with Watson OpenScale standard.

In [78]:
values = []

for v in payload.split('\n'):
    values.append([float(s) for s in v.split(',')])

request_data = {'fields': fields, 'values': values}

response_data = {'fields': list(result['predictions'][0]),
            'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [79]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=request_data, response=response_data, response_time=response_time))

subscription.payload_logging.store(records=records_list)

#### Store the payload using REST API

Get the token first.

In [80]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":WATSON_OS_CREDENTIALS["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

Store the payload.

In [81]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(WATSON_OS_CREDENTIALS['url'], WATSON_OS_CREDENTIALS['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response_data,
    'request': request_data
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

Request OK: True


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [82]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=5)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of the feedback table using below method.

In [83]:
subscription.feedback_logging.print_table_schema()

radius_mean,double,True
texture_mean,double,True
perimeter_mean,double,True
area_mean,double,True
smoothness_mean,double,True
compactness_mean,double,True
concavity_mean,double,True
concave points_mean,double,True
symmetry_mean,double,True
fractal_dimension_mean,double,True
radius_se,double,True


The feedback records can be send to feedback table using the code below.

In [84]:
feedback_records = []

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'diagnosis']

for i in range(1, 10):
    feedback_records.append([17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482, 1])

subscription.feedback_logging.store(feedback_data=feedback_records, fields=fields)

### Run quality monitoring on demand

By default, quality monitoring is run on an hourly schedule. You can also trigger it on demand using the code below.

In [85]:
run_details = subscription.quality_monitoring.run()

Since the monitoring runs in the background you can use the method below to check the status of the job.

In [86]:
status = run_details['status']
id = run_details['id']

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

Run status: initializing
Run status: completed


### Show the quality metrics

In [87]:
subscription.quality_monitoring.show_table()

2019-01-28 16:47:38.105000+00:00,1.0,0.7,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,Accuracy_evaluation_01d14a4d-4dd1-46d5-9906-8a112f01bdc4,


Get all calculated metrics.

In [88]:
deployment_uids = subscription.get_deployment_uids()

In [89]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uids[0])

{'start': '2019-01-28T15:46:46.542Z',
 'end': '2019-01-28T16:51:54.775258Z',
 'metrics': [{'timestamp': '2019-01-28T16:47:38.105Z',
   'value': {'metrics': [{'name': 'weightedTruePositiveRate', 'value': 1.0},
     {'name': 'accuracy', 'value': 1.0},
     {'name': 'weightedFMeasure', 'value': 1.0},
     {'name': 'weightedRecall', 'value': 1.0},
     {'name': 'weightedFalsePositiveRate', 'value': None},
     {'name': 'weightedPrecision', 'value': 1.0}],
    'quality': 1.0,
    'threshold': 0.7},
   'process': 'Accuracy_evaluation_01d14a4d-4dd1-46d5-9906-8a112f01bdc4'}]}

<a id="datamart"></a>
## 6. Get the logged data

### 6.1 Payload logging

#### Print schema of payload_logging table

In [90]:
subscription.payload_logging.print_table_schema()

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
radius_mean,double,True
texture_mean,double,True
perimeter_mean,double,True
area_mean,double,True
smoothness_mean,double,True
compactness_mean,double,True
concavity_mean,double,True


#### Show (preview) the table

In [91]:
subscription.payload_logging.describe_table()

       radius_mean  texture_mean  perimeter_mean     area_mean  \
count        24.00         24.00            24.0  2.400000e+01   
mean         17.02         23.98           112.8  8.993000e+02   
std           0.00          0.00             0.0  1.161320e-13   
min          17.02         23.98           112.8  8.993000e+02   
25%          17.02         23.98           112.8  8.993000e+02   
50%          17.02         23.98           112.8  8.993000e+02   
75%          17.02         23.98           112.8  8.993000e+02   
max          17.02         23.98           112.8  8.993000e+02   

       smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
count     2.400000e+01           24.0000         24.0000              24.0000   
mean      1.197000e-01            0.1496          0.2417               0.1203   
std       1.417627e-17            0.0000          0.0000               0.0000   
min       1.197000e-01            0.1496          0.2417               0.1203   


#### Return the table content as pandas dataframe

In [92]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

### 6.2 Feddback logging

Check the schema of the table.

In [93]:
subscription.feedback_logging.print_table_schema()

radius_mean,double,True
texture_mean,double,True
perimeter_mean,double,True
area_mean,double,True
smoothness_mean,double,True
compactness_mean,double,True
concavity_mean,double,True
concave points_mean,double,True
symmetry_mean,double,True
fractal_dimension_mean,double,True
radius_se,double,True


Preview the table content.

In [94]:
subscription.feedback_logging.show_table()

17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00
17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344.0,0.1634,0.3559,0.5588,0.1847,0.353,0.08482,1.0,2019-01-28 16:47:33.973000+00:00


Describe the table (calulcate basic statistics).

In [95]:
subscription.feedback_logging.describe_table()

       radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
count         9.00          9.00             9.0        9.0     9.000000e+00   
mean         17.02         23.98           112.8      899.3     1.197000e-01   
std           0.00          0.00             0.0        0.0     1.471962e-17   
min          17.02         23.98           112.8      899.3     1.197000e-01   
25%          17.02         23.98           112.8      899.3     1.197000e-01   
50%          17.02         23.98           112.8      899.3     1.197000e-01   
75%          17.02         23.98           112.8      899.3     1.197000e-01   
max          17.02         23.98           112.8      899.3     1.197000e-01   

       compactness_mean  concavity_mean  concave points_mean  symmetry_mean  \
count            9.0000          9.0000               9.0000         9.0000   
mean             0.1496          0.2417               0.1203         0.2248   
std              0.0000          0.0000   

Get the table content.

In [96]:
feedback_pd = subscription.feedback_logging.get_table_content(format='pandas')

### 6.3 Quality metrics table

In [97]:
subscription.quality_monitoring.print_table_schema()

ts,timestamp,False
quality,float,False
quality_threshold,float,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [98]:
subscription.quality_monitoring.show_table()

2019-01-28 16:47:38.105000+00:00,1.0,0.7,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,Accuracy_evaluation_01d14a4d-4dd1-46d5-9906-8a112f01bdc4,


### 6.4 Performance metrics table

In [99]:
subscription.performance_monitoring.print_table_schema()

ts,timestamp,False
scoring_time,float,False
scoring_records,object,False
binding_id,string,False
subscription_id,string,False
deployment_id,string,True
process,string,False
asset_revision,string,True


In [100]:
subscription.performance_monitoring.show_table()

2019-01-28 16:47:14.699834+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699933+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699817+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699917+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699772+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699884+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699850+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699799+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699900+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,
2019-01-28 16:47:14.699723+00:00,462.0,2,152fdb6f-4c8c-43cc-b2e4-73cf388844ef,70b2a1a8cb6b8950ffd2b65afb9236de,865ac8c78c11d062c76138af425834ca,,


### 6.5 Data Mart measurement facts table

In [101]:
client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'subscription': {'subscription_id': '70b2a1a8cb6b8950ffd2b65afb9236de',
    'url': '/v1/data_marts/b873054a-9264-48c4-bcfe-c462ac3b8cf8/service_bindings/152fdb6f-4c8c-43cc-b2e4-73cf388844ef/subscriptions/70b2a1a8cb6b8950ffd2b65afb9236de'},
   'asset': {'name': 'Breast-cancer-linear-learner-2019-01-28-16-13',
    'asset_id': '70b2a1a8cb6b8950ffd2b65afb9236de',
    'url': 's3://scottda-sagemaker-us-east-2/breast-cancer/output/Breast-cancer-linear-learner-2019-01-28-16-13/output/model.tar.gz',
    'asset_type': 'model',
    'created_at': '2019-01-28T16:19:02.652Z'},
   'deployment': {'name': 'Breast-cancer-endpoint-scoring-2019-01-28-16-13',
    'url': 'Breast-cancer-endpoint-scoring-2019-01-28-16-13',
    'deployment_type': 'online',
    'scoring_endpoint': {'url': 'Breast-cancer-endpoint-scoring-2019-01-28-16-13',
     'request_headers': {'Content-Type': 'application/json'}},
    'deployment_rn': 'arn:aws:sagemaker:us-east-2:014862798213:endpoint/breast-cancer-e

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.